In [1]:
import re

import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader

from models.model_definition import get_model, get_tokenizer
from models.lit_punctuator import LitPunctuator
from models.lit_two_head import LitTwoHead
from data_processing.lentaset import LentaSet
%load_ext autoreload
%autoreload 2
from config import config, char2label
from utils.initialization import seed_everything

In [2]:
# config

{'dataset_path': '/home/alidar/MIPT/datasets/lenta-ru-news.csv',
 'sequence_len': 256,
 'dev_mode': True,
 'chunk_size': 50000,
 'accelerator': 'gpu',
 'model': 'two_head',
 'encoder': 'cointegrated/rubert-tiny2',
 'use_crf': True,
 'augment_rate': 0.3,
 'ckpt_path': '/home/alidar/MIPT/punctation_restoration/checkpoints/1pgs7z8i/last.ckpt',
 'continue_from_ckpt': False,
 'epochs': 50,
 'batch_size': 64,
 'num_workers': 1,
 'log_every_n_step': 10,
 'val_check_interval': 1.0,
 'wandb': {'project': 'punctuation_restoration',
  'name': 'two_head_crf',
  'mode': 'online'},
 'seed': 42}

In [3]:
SEED = config["seed"]
MODEL = config["model"]
USE_CRF = config["use_crf"]
ENCODER = config["encoder"]
DATASET_PATH = config["dataset_path"]
SEQUENCE_LEN = config["sequence_len"]
AUGMENT_RATE = config["augment_rate"]
BATCH_SIZE = config["batch_size"]
NUM_WORKERS = config["num_workers"]
DEV_MODE = config["dev_mode"]
CHUNK_SIZE = config["chunk_size"]
CKPT_PATH = config["ckpt_path"]

In [4]:
seed_everything(SEED)
model = None
model, is_two_head = get_model(MODEL, ENCODER, USE_CRF)
tokenizer = get_tokenizer(ENCODER)
if is_two_head:
    lit_model = LitTwoHead(model, tokenizer, USE_CRF)
else:
    lit_model = LitPunctuator(model, tokenizer, USE_CRF)

Global seed set to 42
Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
trainer = pl.Trainer(gpus=0, accelerator="cpu", devices=1,)

/home/alidar/MIPT/punctation_restoration/venv/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/alidar/MIPT/punctation_restoration/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1812: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [6]:
text = """
Епархия была образована 2 ноября 1949 года путем выделения в новую епархию некоторых приходских округов из епархий Сантандера, Витории и Калаорры — Ла-Кальсады. Латинское название епархии — Dioecesis Flaviobrigensis было присвоено епархии в булле папы Пия XII Quo Commodius, которой он учреждал епархию. Вероятно, оно связано с предположением, что римское поселение «Colonia Flaviobrigensium» находилось на месте современного Бильбао, что однако ныне считается маловероятным.
"""

In [7]:
text = """
Активно занимался спортом. Был кандидатом в мастера спорта по лыжному спорту и лёгкой атлетике и первый разряд по футболу и хоккею. Затем увлёкся борьбой. Становился победителем и призёром многих республиканских, ведомственных и всероссийский соревнований. В 1967 году выполнил норматив мастера спорта СССР по самбо. В 1968 году стал серебряным призёром чемпионата Татарской АССР по вольной борьбе и бронзовым призёром чемпионата России по борьбе куреш. В 1971 году стал мастером спорта по дзюдо. В 1975—1977 годах признавался лучшим тренером Татарской АССР. В 1983 году ему было присвоено звание Заслуженного тренера РСФСР, а в 1994 году — звание Заслуженного работника физической культуры республики Татарстан. В 2004 году стал чемпионом России и Европы по дзюдо среди ветеранов и бронзовым призёром чемпионата мира по дзюдо среди ветеранов. Тогда же ему было присвоено звание мастера спорта России международного класса. 
"""

In [10]:
text = """
Шпажник, или Гладиóлус (лат. Gladíolus) — род многолетних клубнелуковичных растений семейства Ирисовые (Iridaceae). Латинское (и русское тоже) название произошло от лат. gladius — «меч» и связано с тем, что листья гладиолуса по форме напоминают шпаги. Природный ареал — тропические и субтропические районы Африки, Средиземноморья, Средние Европа и Азия, Западная Сибирь. Род включает около 280 видов, из которых 163 происходят из южной части Африки, 10 из Евразии, 9 произрастают на Мадагаскаре. 
"""

In [11]:
dataset = LentaSet([text], tokenizer, SEQUENCE_LEN, AUGMENT_RATE, is_train=True)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=NUM_WORKERS)
result = trainer.predict(model=lit_model, dataloaders=dataloader, ckpt_path=CKPT_PATH)[0]

print()
print("Оригинал:")
print(text)
print("Результат:")
print(result)

Restoring states from the checkpoint path at /home/alidar/MIPT/punctation_restoration/checkpoints/1pgs7z8i/last.ckpt
Loaded model weights from checkpoint at /home/alidar/MIPT/punctation_restoration/checkpoints/1pgs7z8i/last.ckpt


Predicting: 0it [00:00, ?it/s]


Оригинал:

Шпажник, или Гладиóлус (лат. Gladíolus) — род многолетних клубнелуковичных растений семейства Ирисовые (Iridaceae). Латинское (и русское тоже) название произошло от лат. gladius — «меч» и связано с тем, что листья гладиолуса по форме напоминают шпаги. Природный ареал — тропические и субтропические районы Африки, Средиземноморья, Средние Европа и Азия, Западная Сибирь. Род включает около 280 видов, из которых 163 происходят из южной части Африки, 10 из Евразии, 9 произрастают на Мадагаскаре. 

Результат:
Шпажник или гладилс (Лат (Gladolus )Род многолетних клубнелуковичных растений семейства ирисовые Iridaceae Латинское и русское тоже название произошло от Лат (Gladius )меч и связано с тем, что листья гладиолуса по форме напоминают шпаги, природный ареал, тропические и субтропические районы Африки, Средиземноморья, Средние Европа и Азия. Западная Сибирь. Род включает около 280 видов из которых 163 происходят из южной части Африки 10 из Евразии. 9 произрастают на Мадагаскаре. 